# Word Embeddings

## Loading Datasets & Importing Modules

In [28]:
import sys
sys.path.append('./Files')
sys.path.append('./Datasets')

In [29]:
import nltk
import gensim
from gensim.models import Word2Vec
import pandas as pd
from Sequencer import Sequencer
from helper_fns import write_dict
from helper_fns import read_dict
import numpy as np
import matplotlib.pyplot as plt
import json
import string

In [30]:
import warnings
warnings.filterwarnings(action = 'ignore')

## Word2Vec

### CBOW

In [31]:
fpath = "Text Preprocessing Experimentations/exp1_rtp_rl_re_cat_rl_qalsadi.csv"
name = "exp1"
mc = 5
hc = 2
vs = 100

In [32]:
df = pd.read_csv(fpath)
df.dropna(inplace=True)

In [33]:
tweets = []
for i in df['tweet']:
    tweet = nltk.word_tokenize(i)
    tweets.append(tweet)

In [34]:
CBoW = Word2Vec(tweets,
                vector_size=vs,
                window=hc,
                min_count=mc)

In [35]:
CBoW_dict = {}
for word in CBoW.wv.key_to_index:
    CBoW_dict[word] = CBoW.wv[word].tolist()

In [36]:
len(CBoW.wv.key_to_index)

3420

In [37]:
CBoW.save("Word Embeddings/cb_{}_{}_{}_{}.model".format(name, mc, hc, vs))

In [38]:
write_dict("Word Embeddings/cb_dict_{}_{}_{}_{}.txt".format(name, mc, hc, vs), CBoW_dict)

### Skip-Gram

In [48]:
fpath = "Text Preprocessing Experimentations/exp1_rtp_rl_re_cat_rl_qalsadi.csv"
name = "exp1"
mc = 5
hc = 2
vs = 100

In [49]:
df = pd.read_csv(fpath)
df.dropna(inplace=True)

In [50]:
tweets = []
for i in df['tweet']:
    tweet = nltk.word_tokenize(i)
    tweets.append(tweet)

In [51]:
SkipGram = Word2Vec(tweets,
                    min_count=mc,
                    window=hc,
                    vector_size=vs,
                    sorted_vocab=1,
                    sg=1)

In [52]:
skipgram_dict = {}
for word in SkipGram.wv.key_to_index:
    skipgram_dict[word] = SkipGram.wv[word].tolist()

In [53]:
len(SkipGram.wv.key_to_index)

3420

In [54]:
SkipGram.save("Word Embeddings/sg_{}_{}_{}_{}.model".format(name, mc, hc, vs))

In [56]:
write_dict("Word Embeddings/sg_dict_{}_{}_{}_{}.txt".format(name, mc, hc, vs), skipgram_dict)

## GloVe